In [1]:
!pip install -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 28.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM , GRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
import math
import seaborn as sns
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from pandas.core.frame import DataFrame
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam
from keras.layers import Conv1D, Conv2D, MaxPooling2D, MaxPooling1D
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import MinMaxScaler
import csv
import collections
from scipy.stats import zscore
from datetime import datetime
import tensorflow as tf
import collections
from scipy.stats import zscore
from datetime import datetime
import pickle
from keras.layers import Embedding, Bidirectional, SimpleRNN

In [ ]:
print(keras.__version__)
print(tf.__version__)

3.8.0
2.18.0


## Required functions

### Company data prepration functions

In [ ]:
# convert history into inputs and outputs
def to_supervised(data, n_input, n_out):
  # flatten data
  X, y = list(), list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(data)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(data):
      X.append(data.iloc[in_start:in_end, :])
      y.append(data.iloc[in_end:out_end, 1])
    # move along one time step
    in_start += 1
  print(f'X shape: {array(X).shape}')
  print(f'y shape: {array(y).shape}')
  return array(X), array(y)

def get_last_val_set_features(data):
  # flatten data
  X = list()
  X.append(data.iloc[len(data)-5:, :])
  return X


In [ ]:
#Test starting from 1-1-2017 based on 'Present Effictive short-term forecasts of Saudi stock price trends using technical indicators and large-scale multivariate time series'
# split a multivariate dataset into train/test sets
def prepare_company_for_training_and_testing(company,selected_ti):
  train,val, test =  company.loc[:'2015-12-31'],company.loc['1-1-2016':'31-12-2016'],company.loc['1-1-2017':]
  print(f'Train shape: {train.shape}, Val shape: {val.shape}, Test shape: {test.shape}')
  scaler = StandardScaler()
  scaler.fit(train)
  data = scaler.transform(train)
  train_normlized = DataFrame(data = data , columns = selected_ti ,index=train.index)
  scaler = StandardScaler()
  scaler.fit(val)
  data = scaler.transform(val)
  val_normlized = DataFrame(data = data , columns = selected_ti ,index=val.index)
  scaler = StandardScaler()
  scaler.fit(test)
  data = scaler.transform(test)
  test_normlized = DataFrame(data = data , columns =selected_ti,index=test.index)
  print("*******************************************")
  print("Train data after multistep approach")
  train_x_normlized, train_y_normlized = to_supervised(train_normlized, 5,5)
  print("Validation multistep approach")
  val_x_normlized,val_y_normlized = to_supervised(val_normlized, 5,5)
  print("Test multistep approach")
  last_val = get_last_val_set_features(val_normlized)
  test_x_normlized,test_y_normlized = to_supervised(test_normlized, 5,5)
  return train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized, test

### Company models creating and evaluation functions

In [ ]:
def true_y_to_evaluation_form(y_true, n_input, n_out):
  y = list()
  in_start = 0
	# step over the entire history one time step at a time
  for _ in range(len(y_true)):
    # define the end of the input sequence
    in_end = in_start + n_input
    out_end = in_end + n_out
    # ensure we have enough data for this instance
    if out_end < len(y_true):
      y.append(y_true[in_end:out_end])
    # move along one time step
    in_start += 1
  return array(y)

# make a forecast
def forecast(model, history):
	yhat = model.predict(history, verbose=0)
	# we only want the vector forecast
	return yhat

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
  scaler = StandardScaler()
  scaler.fit(actual)
  predicted_not_normlized = scaler.inverse_transform(predicted)
  mse_score= mse(actual,predicted_not_normlized)
  rmse_score = sqrt(mse_score)
  mae_score = mae(actual,predicted_not_normlized)
  final_r2_score = r2_score(actual,predicted_not_normlized,multioutput='variance_weighted')
  return mse_score, rmse_score,mae_score,final_r2_score,predicted_not_normlized

# summarize scores
def summarize_scores(name, mse_score, rmse_score,mae_score,final_r2_score):
  #s_scores = ', '.join(['%.1f' % s for s in scores])
  #print('%s: [%.3f] %s' % (name, score, s_scores))
  print(f'*************{name}**********')
  print("MAE: ",mae_score)
  print("MSE: ",mse_score)
  print("RMSE: ",rmse_score)
  print("R2: ",final_r2_score)

# evaluate a single model
def evaluate_model(test_x,test_y,model):
	# walk-forward validation over each week
  predictions = list()
  predictions = forecast(model, test_x)
  # evaluate predictions days for each week
  predictions = array(predictions)
  mse_score, rmse_score,mae_score,final_r2_score, y_hat_not_normlized = evaluate_forecasts(test_y, predictions)
  return mse_score, rmse_score,mae_score,final_r2_score,y_hat_not_normlized

In [ ]:
def get_brnn_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df):
  # define parameters
  verbose, epochs, batch_size = 0, 32, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

  i = 2
  while i < 1030:
    # define model
    model_BRNN_prposed = Sequential()
    model_BRNN_prposed.add(Bidirectional(SimpleRNN(i, activation='relu', input_shape=(n_timesteps, n_features))))
    model_BRNN_prposed.add(Dense(5))
    model_BRNN_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_BRNN_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_BRNN_prposed)
    evaluation_df.loc[i,"MAE_b"],evaluation_df.loc[i,"MSE_b"],evaluation_df.loc[i,"RMSE_b"],evaluation_df.loc[i,"R2_b"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of BRNN_ is {i}")
    pickle.dump(model_BRNN_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_BRNN_{i}.pkl' , 'wb'))
    summarize_scores('Prposed BRNN_', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"BRNN_{i}"] = predictions_L_p.flatten()
    i = i * 2

  i = 3
  while i < 1030:
    # define model
    model_BRNN_prposed = Sequential()
    model_BRNN_prposed.add(Bidirectional(SimpleRNN(i, activation='relu', input_shape=(n_timesteps, n_features))))
    model_BRNN_prposed.add(Dense(5))
    model_BRNN_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_BRNN_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_BRNN_prposed)
    evaluation_df.loc[i,"MAE_b"],evaluation_df.loc[i,"MSE_b"],evaluation_df.loc[i,"RMSE_b"],evaluation_df.loc[i,"R2_b"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of BRNN_ is {i}")
    pickle.dump(model_BRNN_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_BRNN_{i}.pkl' , 'wb'))
    summarize_scores('Prposed BRNN_', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"BRNN_{i}"] = predictions_L_p.flatten()
    i = i * 3

  i = 100
  while i < 1030:
    # define model
    model_BRNN_prposed = Sequential()
    model_BRNN_prposed.add(Bidirectional(SimpleRNN(i, activation='relu', input_shape=(n_timesteps, n_features))))
    model_BRNN_prposed.add(Dense(5))
    model_BRNN_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_BRNN_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_BRNN_prposed)
    evaluation_df.loc[i,"MAE_b"],evaluation_df.loc[i,"MSE_b"],evaluation_df.loc[i,"RMSE_b"],evaluation_df.loc[i,"R2_b"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of BRNN_ is {i}")
    pickle.dump(model_BRNN_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_BRNN_{i}.pkl' , 'wb'))
    summarize_scores('Prposed BRNN_', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"BRNN_{i}"] = predictions_L_p.flatten()
    i = i + 100

  return evaluation_df,predictions_df

def get_gru_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df):
  # define parameters
  verbose, epochs, batch_size = 0, 32, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

  i = 2
  while i < 1030:
    # define model
    model_GRU_prposed = Sequential()
    model_GRU_prposed.add(GRU(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_GRU_prposed.add(Dense(5))
    model_GRU_prposed.compile(loss='mse', optimizer='adam')
    tf.keras.utils.plot_model(model_GRU_prposed)
    # fit network
    model_GRU_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,model_GRU_prposed)
    evaluation_df.loc[i,"MAE_g"],evaluation_df.loc[i,"MSE_g"],evaluation_df.loc[i,"RMSE_g"],evaluation_df.loc[i,"R2_g"] = mae_score_G_p,mse_score_G_p, rmse_score_G_p,final_r2_score_G_p
    print(f"Parameter of GRU is {i}")
    pickle.dump(model_GRU_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_GRU_{i}.pkl' , 'wb'))
    summarize_scores('Prposed GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p)
    predictions_df[f"GRU{i}"] = predictions_G_p.flatten()
    i = i * 2

  i = 3
  while i < 1030:
    # define model
    model_GRU_prposed = Sequential()
    model_GRU_prposed.add(GRU(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_GRU_prposed.add(Dense(5))
    model_GRU_prposed.compile(loss='mse', optimizer='adam')
    tf.keras.utils.plot_model(model_GRU_prposed)
    # fit network
    model_GRU_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,model_GRU_prposed)
    evaluation_df.loc[i,"MAE_g"],evaluation_df.loc[i,"MSE_g"],evaluation_df.loc[i,"RMSE_g"],evaluation_df.loc[i,"R2_g"] = mae_score_G_p,mse_score_G_p, rmse_score_G_p,final_r2_score_G_p
    print(f"Parameter of GRU is {i}")
    pickle.dump(model_GRU_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_GRU_{i}.pkl' , 'wb'))
    summarize_scores('Prposed GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p)
    predictions_df[f"GRU{i}"] = predictions_G_p.flatten()
    i = i * 3
  i = 100
  while i < 1030:
    # define model
    model_GRU_prposed = Sequential()
    model_GRU_prposed.add(GRU(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_GRU_prposed.add(Dense(5))
    model_GRU_prposed.compile(loss='mse', optimizer='adam')
    tf.keras.utils.plot_model(model_GRU_prposed)
    # fit network
    model_GRU_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p,predictions_G_p = evaluate_model(test_x,test_y_for_evaluation,model_GRU_prposed)
    evaluation_df.loc[i,"MAE_g"],evaluation_df.loc[i,"MSE_g"],evaluation_df.loc[i,"RMSE_g"],evaluation_df.loc[i,"R2_g"] = mae_score_G_p,mse_score_G_p, rmse_score_G_p,final_r2_score_G_p
    print(f"Parameter of GRU is {i}")
    pickle.dump(model_GRU_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_GRU_{i}.pkl' , 'wb'))
    summarize_scores('Prposed GRU', mse_score_G_p, rmse_score_G_p,mae_score_G_p,final_r2_score_G_p)
    predictions_df[f"GRU{i}"] = predictions_G_p.flatten()
    i = i + 100
  return evaluation_df,predictions_df

def get_lstm_models_info(train_x,train_y,val_x,val_y,test_x,y_test,test_y_for_evaluation,company_name,evaluation_df,predictions_df):
  # define parameters
  verbose, epochs, batch_size = 0, 32, math.floor(len(train_x)*0.05)
  n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
  # reshape output into [samples, timesteps, features]
  train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

  i = 2
  while i < 1030:
    # define model
    model_LSTM_prposed = Sequential()
    model_LSTM_prposed.add(LSTM(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_LSTM_prposed.add(Dense(5))
    model_LSTM_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_LSTM_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_LSTM_prposed)
    evaluation_df.loc[i,"MAE_l"],evaluation_df.loc[i,"MSE_l"],evaluation_df.loc[i,"RMSE_l"],evaluation_df.loc[i,"R2_l"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of LSTM is {i}")
    pickle.dump(model_LSTM_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_LSTM_{i}.pkl' , 'wb'))
    summarize_scores('Prposed LSTM', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"LSTM_{i}"] = predictions_L_p.flatten()
    i = i * 2

  i = 3
  while i < 1030:
    # define model
    model_LSTM_prposed = Sequential()
    model_LSTM_prposed.add(LSTM(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_LSTM_prposed.add(Dense(5))
    model_LSTM_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_LSTM_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_LSTM_prposed)
    evaluation_df.loc[i,"MAE_l"],evaluation_df.loc[i,"MSE_l"],evaluation_df.loc[i,"RMSE_l"],evaluation_df.loc[i,"R2_l"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of LSTM is {i}")
    pickle.dump(model_LSTM_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_LSTM_{i}.pkl' , 'wb'))
    summarize_scores('Prposed LSTM', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"LSTM_{i}"] = predictions_L_p.flatten()
    i = i * 3

  i = 100
  while i < 1030:
    # define model
    model_LSTM_prposed = Sequential()
    model_LSTM_prposed.add(LSTM(i, activation='relu', input_shape=(n_timesteps, n_features)))
    model_LSTM_prposed.add(Dense(5))
    model_LSTM_prposed.compile(loss='mse', optimizer='adam')
    # fit network
    model_LSTM_prposed.fit(train_x,
            train_y,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(val_x,val_y))
    mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p,predictions_L_p = evaluate_model(test_x,test_y_for_evaluation,model_LSTM_prposed)
    evaluation_df.loc[i,"MAE_l"],evaluation_df.loc[i,"MSE_l"],evaluation_df.loc[i,"RMSE_l"],evaluation_df.loc[i,"R2_l"] = mae_score_L_p,mse_score_L_p, rmse_score_L_p,final_r2_score_L_p
    print(f"Parameter of LSTM is {i}")
    pickle.dump(model_LSTM_prposed , open(f'/content/drive/MyDrive/KFUPM/Graduation/final_experiments/models/US/{company_name}_LSTM_{i}.pkl' , 'wb'))
    summarize_scores('Prposed LSTM', mse_score_L_p, rmse_score_L_p,mae_score_L_p,final_r2_score_L_p)
    predictions_df[f"LSTM_{i}"] = predictions_L_p.flatten()
    i = i + 100

  return evaluation_df,predictions_df

In [ ]:
def train_evalute_models_for_company(train_x,train_y,val_x,val_y,test_x,y_test,test,company_name):
  eval_cols = ['MAE_b','MSE_b','RMSE_b','R2_b','MAE_g','MSE_g','RMSE_g','R2_g','MAE_l','MSE_l','RMSE_l','R2_l']
  unit_indices = [2,4,8,16,32,64,128,256,512,1024,3,9,27,81,243,729,100,200,300,400,500,600,700,800,900,1000]
  test_y_for_evaluation = true_y_to_evaluation_form(test['Close'].values, 5, 5)
  evaluation_df = pd.DataFrame(columns = eval_cols, index = unit_indices)
  predictions_df = DataFrame(data = test_y_for_evaluation.flatten())
  #LSTM
  evaluation_df,predictions_df = get_lstm_models_info(train_x, train_y, val_x, val_y, test_x, y_test, test_y_for_evaluation, company_name, evaluation_df, predictions_df)
  #BRNN
  evaluation_df,predictions_df = get_brnn_models_info(train_x, train_y, val_x, val_y, test_x, y_test, test_y_for_evaluation, company_name, evaluation_df, predictions_df)
  #GRU
  evaluation_df,predictions_df = get_gru_models_info(train_x, train_y, val_x, val_y, test_x, y_test, test_y_for_evaluation, company_name, evaluation_df, predictions_df)
  return evaluation_df,predictions_df



## Excution

### AAPL

In [ ]:
aapl = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/US/AAPL_selected_features.csv',index_col='Unnamed: 0',parse_dates=True)
aapl.head()

,Adjusted Close,Close,High,Low,Open,alma,decay,dema,ema,fwma,...,sma,swma,t3,tema,trima,vmwa,vwap,wcp,wma,zlma
2001-12-31,0.333356,0.391071,0.404643,0.389821,0.401964,0.0,0.391071,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.395178,0.394151,0.000000,0.000000
2002-01-01,0.333356,0.391071,0.404643,0.389821,0.401964,0.0,0.391071,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.395178,0.394151,0.000000,0.000000
2002-01-02,0.354666,0.416071,0.416071,0.392143,0.393750,0.0,0.416071,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.408095,0.410089,0.000000,0.000000
2002-01-03,0.358928,0.421071,0.424107,0.406607,0.410714,0.0,0.421071,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.417262,0.418214,0.000000,0.000000
2002-01-04,0.360603,0.423036,0.427679,0.410536,0.416786,0.0,0.423036,0.408464,0.408464,0.416056,...,0.408464,0.409623,0.408464,0.408464,0.409623,0.415538,0.420417,0.421072,0.414726,0.440714


In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(aapl, aapl.columns)

Train shape: (5114, 33), Val shape: (366, 33), Test shape: (1160, 33)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 33)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 33)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 33)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized, train_y_normlized, val_x_normlized, val_y_normlized, test_x_normlized, test_y_normlized, test, 'AAPL')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 2
*************Prposed LSTM**********
MAE:  1.1679950126183551
MSE:  2.5872429513331396
RMSE:  1.608490892524151
R2:  0.9788809349533486


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  1.6701763867850896
MSE:  6.28124956050033
RMSE:  2.5062421192894213
R2:  0.948727614476985


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Parameter of GRU is 800
*************Prposed GRU**********
MAE:  0.8370667989581568
MSE:  1.5699661791298396
RMSE:  1.2529829125450354
R2:  0.9871847296594233


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 900
*************Prposed GRU**********
MAE:  0.90652915900189
MSE:  1.6992085745103083
RMSE:  1.3035369478884395
R2:  0.9861297539164536


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.8350756546551252
MSE:  1.5894830564132583
RMSE:  1.2607470231625606
R2:  0.9870254179099627


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/AAPL_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/AAPL_predictions.csv")

### BAC

In [ ]:
bac = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/US/BAC_selected_features.csv',index_col='Unnamed: 0',parse_dates=True)
bac.head()

,Adjusted Close,Close,High,Low,Open,alma,decay,dema,ema,fwma,...,sma,swma,t3,tema,trima,vmwa,vwap,wcp,wma,zlma
2001-12-31,18.983713,31.475,31.875,31.410,31.875,0.0,31.475000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.586670,31.55875,0.000000,0.00000
2002-01-01,18.983713,31.475,31.875,31.410,31.875,0.0,31.475000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.586670,31.55875,0.000000,0.00000
2002-01-02,18.986729,31.480,31.550,31.055,31.155,0.0,31.480000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.361664,31.39125,0.000000,0.00000
2002-01-03,18.965622,31.445,31.950,31.380,31.480,0.0,31.445000,0.000,0.000,0.00,...,0.000,0.000000,0.000,0.000,0.000000,0.000000,31.591667,31.55500,0.000000,0.00000
2002-01-04,19.261154,31.935,32.075,31.505,31.560,0.0,31.934999,31.562,31.562,31.66,...,31.562,31.521111,31.562,31.562,31.521111,31.596972,31.838333,31.86250,31.621333,31.76333


In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(bac, bac.columns)

Train shape: (5114, 33), Val shape: (366, 33), Test shape: (1160, 33)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 33)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 33)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 33)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,"BAC")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 2
*************Prposed LSTM**********
MAE:  0.8269072660628606
MSE:  1.4238510666211506
RMSE:  1.1932523063548424
R2:  0.8692963328227198


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  1.3665096161552082
MSE:  3.042635790546274
RMSE:  1.7443152784248248
R2:  0.7206985582748071


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Parameter of GRU is 800
*************Prposed GRU**********
MAE:  0.48445627161440663
MSE:  0.49361428322550427
RMSE:  0.7025768877678117
R2:  0.9546882405743743


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 900
*************Prposed GRU**********
MAE:  0.4898567186173152
MSE:  0.49830574656153137
RMSE:  0.7059077464949165
R2:  0.9542575835507423


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.47862991634203056
MSE:  0.487512367053813
RMSE:  0.6982208583634644
R2:  0.9552483713627306


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/BAC_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/BAC_predictions.csv")

### J&J

In [ ]:
jnj = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/US/JNJ_selected_features.csv',index_col='Unnamed: 0',parse_dates=True)
jnj.head()

,Adjusted Close,Close,High,Low,Open,alma,decay,dema,ema,fwma,...,sma,swma,t3,tema,trima,vmwa,vwap,wcp,wma,zlma
2001-12-31,33.781322,59.10,59.82,58.88,59.70,0.0,59.099998,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.266663,59.2250,0.000,0.000000
2002-01-01,33.781322,59.10,59.82,58.88,59.70,0.0,59.099998,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,59.266663,59.2250,0.000,0.000000
2002-01-02,33.552654,58.70,59.30,57.58,58.90,0.0,58.899998,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,58.526670,58.5700,0.000,0.000000
2002-01-03,33.541237,58.68,58.73,57.88,58.70,0.0,58.680000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,58.430008,58.4925,0.000,0.000000
2002-01-04,33.232563,58.14,58.90,57.95,58.68,0.0,58.480000,58.743996,58.743996,58.528333,...,58.743999,58.766666,58.743996,58.743996,58.766666,58.740904,58.330006,58.2825,58.588,58.046665


In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(jnj, jnj.columns)

Train shape: (5114, 33), Val shape: (366, 33), Test shape: (1160, 33)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 33)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 33)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 33)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,"JNJ")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 2
*************Prposed LSTM**********
MAE:  3.3985592595108662
MSE:  23.38963241816463
RMSE:  4.836282913371035
R2:  0.6280258224056705


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  2.6882014880901863
MSE:  14.905206764662335
RMSE:  3.86072619654157
R2:  0.762956855027234


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Parameter of GRU is 800
*************Prposed GRU**********
MAE:  1.6030605800462816
MSE:  5.52472430004258
RMSE:  2.3504732076844825
R2:  0.9121382182839366


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 900
*************Prposed GRU**********
MAE:  1.5338669144870927
MSE:  5.156389068812757
RMSE:  2.270768387311387
R2:  0.9179959928853588


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  1.5287625082795517
MSE:  5.048925086552741
RMSE:  2.2469813275932538
R2:  0.9197050332716087


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/jnj_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/jnj_predictions.csv")

### GOOG

In [ ]:
goog = pd.read_csv('/content/drive/MyDrive/KFUPM/Graduation/data/US/GOOG_selected_features.csv',index_col='Unnamed: 0',parse_dates=True)
goog.head()

,Adjusted Close,Close,High,Low,Open,alma,decay,dema,ema,fwma,...,sma,swma,t3,tema,trima,vmwa,vwap,wcp,wma,zlma
2001-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2002-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test = prepare_company_for_training_and_testing(goog, goog.columns)

Train shape: (5114, 33), Val shape: (366, 33), Test shape: (1160, 33)
*******************************************
Train data after multistep approach
X shape: (5104, 5, 33)
y shape: (5104, 5)
Validation multistep approach
X shape: (356, 5, 33)
y shape: (356, 5)
Test multistep approach
X shape: (1150, 5, 33)
y shape: (1150, 5)


In [ ]:
evaluation_df,predictions_df = train_evalute_models_for_company(train_x_normlized,train_y_normlized,val_x_normlized,val_y_normlized,test_x_normlized,test_y_normlized,test,"GOOG")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 2
*************Prposed LSTM**********
MAE:  1.5161969663245154
MSE:  4.605122914327984
RMSE:  2.145955012186412
R2:  0.9238986370454517


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of LSTM is 4
*************Prposed LSTM**********
MAE:  2.2524368689070973
MSE:  15.449770825090713
RMSE:  3.9306196489981975
R2:  0.7446868109716043


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Parameter of GRU is 800
*************Prposed GRU**********
MAE:  0.8950693546301807
MSE:  1.545973924128192
RMSE:  1.2433719974843378
R2:  0.9744522079199455


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 900
*************Prposed GRU**********
MAE:  0.8459972917745846
MSE:  1.469687938349605
RMSE:  1.212306866411968
R2:  0.9757128621087877


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Parameter of GRU is 1000
*************Prposed GRU**********
MAE:  0.8567046749055288
MSE:  1.4828662694257322
RMSE:  1.217729965725461
R2:  0.975495085303474


In [ ]:
evaluation_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/goog_evaluation.csv")

In [ ]:
predictions_df.to_csv(f"/content/drive/MyDrive/KFUPM/Graduation/final_experiments/results/goog_predictions.csv")